<a href="https://colab.research.google.com/github/aswinaus/Reinforcement-Learning/blob/main/rl_deepseekr1_awq_quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers torch accelerate langchain langchain_huggingface

Code is essentially forcing Python to always use "UTF-8" as the preferred encoding, regardless of the user's actual system settings. UTF-8 is a widely used encoding that can represent a vast range of characters from different languages. By enforcing UTF-8, you can help ensure that your code works consistently across different platforms and avoids encoding-related errors. It's a common practice for improving compatibility and preventing issues with text handling in Python programs.

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [4]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Agents` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Agents`


In [5]:
from google.colab import drive
drive.mount('/content/drive')
# Download Data
data_dir = '/content/drive/MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread
device = 'cuda' if torch.cuda.is_available() else 'cpu'

The nvidia-smi command is a utility provided by NVIDIA to query and display information about your NVIDIA GPU(s) (Graphics Processing Unit). This includes things like:

GPU model and name
Driver version
GPU utilization
Memory usage
Temperature
Power consumption
Processes running on the GPU

In [7]:
!nvidia-smi

Wed Mar 12 19:05:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
!pip install autoawq
from typing import Tuple, Optional, Union, Dict, Any
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, AutoConfig
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

In [8]:
model_path = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
#model.quantize(tokenizer, quant_config=quant_config)

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

benchmark.jpg:   0%|          | 0.00/777k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from google.colab import drive
drive.mount('/content/drive')
# Download Data
data_dir = '/content/drive/MyDrive'
model_path = 'deepseekr1/DeepSeek-R1-Distill-Qwen-7B'
quant_path = f"/{data_dir}/LLMs/Deepseek/DeepSeek-R1-Distill-Qwen-7B"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#Save quantized model
#model.save_quantized(quant_path)
#tokenizer.save_pretrained(quant_path)
print(f'Model is quantized and saved at "{quant_path}"')

Model is quantized and saved at "//content/drive/MyDrive/LLMs/Deepseek/DeepSeek-R1-Distill-Qwen-7B"


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline # Changed to AutoModelForCausalLM, imported pipeline
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline

In [12]:
local_model_path = quant_path
local_tokenizer = AutoTokenizer.from_pretrained(quant_path)
local_model = AutoModelForCausalLM.from_pretrained(quant_path,low_cpu_mem_usage=True)

You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "How does DeepSeek-R1 prove it uses reinforcement learning?"},
]
pipe = pipeline("text-generation", model=local_model, tokenizer=local_tokenizer)
pipe(messages)

Device set to use cuda:0


[{'generated_text': [{'role': 'user',
    'content': 'How does DeepSeek-R1 prove it uses reinforcement learning?'},
   {'role': 'assistant',
    'content': "Hi! I'm DeepSeek-R1, an AI assistant independently developed by the Chinese company DeepSeek"}]}]

In [17]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("aswinaus/tax_statistics_dataset_by_income_range", download_mode="force_redownload")

# Get column names from the 'features' of the 'train' split
original_columns = list(dataset['train'].features) # Get original column names

# Create new column names, replacing spaces with underscores only if needed
new_columns = [col.replace(' ', '_') if ' ' in col else col for col in original_columns]

# Rename columns only if the new name is different
for old_col, new_col in zip(original_columns, new_columns):
    if old_col != new_col:  # Only rename if the new name is different
        dataset['train'] = dataset['train'].rename_column(old_col, new_col)

df=pd.DataFrame(dataset['train'])
df.head(10)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

(…)x_statistics_dataset_by_income_range.csv:   0%|          | 0.00/272k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/303 [00:00<?, ? examples/s]

,STATEFIPS,STATE,zipcode,Size_of_adjusted_gross_income,No_of_returns,No_of_single_returns,No_of_joint_returns,No_of_head_of_household_returns,Number_of_electronically_filed_returns,Number_of_computer_prepared_paper_returns,...,Number_of_returns_with_net_investment_income_tax,Net_investment_income_tax_amount,Number_of_returns_with_tax_due_at_time_of_filing,Tax_due_at_time_of_filing_amount,Number_of_returns_with_total_overpayments,Total_overpayments_amount,Number_of_returns_with_overpayments_refunded,Overpayments_refunded_amount,Number_of_returns_with_credit_to_next_years_estimated_tax,Credited_to_next_years_estimated_tax_amount
0,1,AL,0,"$1 under $25,000",778210,491030,84770,189600,712890,30670,...,0,0,62720,51936,671860,1700965,669570,1694792,1980,3512
1,1,AL,0,"$25,000 under $50,000",525940,247140,123910,139860,481760,18960,...,0,0,85860,122569,438020,1274802,435210,1266557,3670,7410
2,1,AL,0,"$50,000 under $75,000",285700,105140,128140,44560,260570,10670,...,0,0,73980,154932,212040,575315,208470,564202,5020,13653
3,1,AL,0,"$75,000 under $100,000",179070,38820,123110,13740,164300,5020,...,0,0,51330,139065,126850,401581,123310,388749,3040,10377
4,1,AL,0,"$100,000 under $200,000",257010,28180,216740,7150,236850,8400,...,90,141,104290,460071,152790,598248,144640,539385,9180,56257
5,1,AL,0,"$200,000 or more",74810,4540,66580,530,69330,1760,...,39430,141243,40030,946610,32060,681565,22420,257633,9450,372747
6,2,AK,0,"$1 under $25,000",102820,81530,7850,10850,89210,6360,...,0,0,17720,8643,75910,133915,75430,132166,370,670
7,2,AK,0,"$25,000 under $50,000",79910,48680,14900,13510,72490,3260,...,0,0,11540,19564,68020,185024,67520,182576,740,1890
8,2,AK,0,"$50,000 under $75,000",51890,25460,17880,6840,46570,2160,...,0,0,11330,26920,40310,118691,39720,115674,830,2922
9,2,AK,0,"$75,000 under $100,000",36350,12880,19390,3190,32620,1630,...,0,0,9690,29647,26340,90982,25670,87439,680,2366


In [19]:
text_columns = ["STATE","Size_of_adjusted_gross_income","No_of_returns","No_of_single_returns","No_of_joint_returns","No_of_head_of_household_returns","Number_of_electronically_filed_returns","Number_of_computer_prepared_paper_returns"]

# Tokenize dataset
def tokenize_all_columns(taxData, text_columns, tokenizer):
    tokenized_data = {}
    for column in taxData:
        if column in text_columns:
            # Initialize an empty list to store tokenized items
            tokenized_list = []
            # Iterate through each item in the column list
            for text in taxData[column]:
                # Ensure text is a string before tokenizing
                if not isinstance(text, str):
                    text = str(text)  # Convert to string if not already
                # Tokenize the single item, adding it to the list.
                tokenized_list.append(tokenizer(text, padding="max_length", truncation=True))
            # Store the tokenized list under the column_tokens key.
            tokenized_data[f"{column}_tokens"] = tokenized_list
            tokenized_data['input_ids'] = [item['input_ids'] for item in tokenized_data['STATE_tokens']]  # Adjust if needed
            tokenized_data['attention_mask'] = [item['attention_mask'] for item in tokenized_data['STATE_tokens']]  # Adjust if needed
    return tokenized_data

tokenized_dataset = dataset.map(
    lambda x: tokenize_all_columns(x, text_columns, local_tokenizer),
    batched=True
)

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

In [ ]:
#Check the tokenized dataframe sample
tokenized_df=pd.DataFrame(tokenized_dataset)
new_columns = [col.replace(' ', '_') for col in tokenized_df.columns]
tokenized_df.columns = new_columns
tokenized_df.head(1)

In [ ]:
#for batch in tokenized_dataset['train']:
#         print(batch.keys())  # Print the keys available in a batch
         # If 'input_ids' is nested, adjust accordingly:
         # input_ids = torch.tensor(batch['some_key']['input_ids']).to(model.device)

In [20]:
import torch

In [21]:
class RewardModel(torch.nn.Module):
    def __init__(self, base_model):
        super(RewardModel, self).__init__()
        self.base_model = base_model
        self.reward_head = torch.nn.Linear(base_model.config.hidden_size, 1)  # One scalar output for reward

    def forward(self, input_ids, attention_mask=None):
        outputs = self.base_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]  # Last layer hidden state
        rewards = self.reward_head(hidden_states.mean(dim=1))  # Average hidden state as input to reward head
        return rewards

# Initialize reward model using the pre-trained model
reward_model = RewardModel(local_model)

In [22]:
reward_model = RewardModel(local_model)

Following line imports the AdamW optimizer from the torch.optim module.

**torch.optim** is a PyTorch library that provides various optimization algorithms used for training machine learning models.

**AdamW** is a specific type of optimizer, a variant of the popular Adam optimizer, that is often preferred for training transformer-based models (like the one being used in the rest of the code).

**Why am I using AdamW instead of the standard Adam optimizer?**It handles weight decay in a slightly different and potentially more effective way than standard Adam.

First create an instance ofthe AdamW optimizer to update the parameters of the reward model essentially to let the optimizers to know which values the optimizer is allowed to adjust to improve the models performance. The value of lr=5e-5 sets the learning rate(lr) for the optimizer to 5e-5(which is 0.00005)

Remember in order to update the Q-value function in RL we need to introduce learning rate which actually is a number between 0 and 1, which can be thought of as how quickly the agent abandons the previous Q-value in the Q-table for a given state-action pair for the new Q-value.

In [23]:
from torch.optim import AdamW
optimizer = AdamW(reward_model.parameters(), lr=5e-5)

In [24]:
# Define the loss function for PPO. This should combine policy loss and reward signal.
def ppo_loss(logits, rewards, baseline=0.0):
    """Compute PPO loss using policy gradients."""
    advantages = rewards - baseline
    policy_loss = -advantages.mean()  # Simplified, normally clip the ratio for PPO
    return policy_loss

In [25]:
def train_ppo(tokenized_dataset, model, reward_model, optimizer, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        for batch in tokenized_dataset:
            input_ids = torch.tensor(batch['input_ids']).to(model.device)
            attention_mask = torch.tensor(batch['attention_mask']).to(model.device)

            # Generate model outputs (logits) for prompts
            outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=False)
            logits = outputs.logits

            # Get reward predictions from the reward model
            rewards = reward_model(input_ids, attention_mask)

            # Compute the PPO loss
            loss = ppo_loss(logits, rewards)

            # Backpropagate and optimize
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print(f"Epoch {epoch+1} completed.")

# Train with PPO
train_ppo(tokenized_dataset['train'], local_model, reward_model, optimizer, num_epochs=1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1792.00 GiB. GPU 0 has a total capacity of 14.74 GiB of which 9.04 GiB is free. Process 40610 has 5.70 GiB memory in use. Of the allocated memory 5.52 GiB is allocated by PyTorch, and 50.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)